In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
# 1.1: Split the dataset into train and test (80:20)
train_dir = 'demo_boat'
test_dir = 'demo_boat'

# 1.2: Initialize ImageDataGenerator with normalization and validation split
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

# 1.4: Load train, validation, and test dataset
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(128, 128),
                                                    batch_size=32, class_mode='categorical',
                                                    subset='training', shuffle=True, seed=43)
val_generator = train_datagen.flow_from_directory(train_dir, target_size=(128, 128),
                                                  batch_size=32, class_mode='categorical',
                                                  subset='validation', shuffle=True, seed=43)
test_generator_tl = test_datagen_tl.flow_from_directory(test_dir, target_size=(128, 128),
                                                        batch_size=32, class_mode='categorical',
                                                        shuffle=False)

In [ ]:
# 1.5: Build CNN model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])


In [ ]:
cnn_model.compile(optimizer=Adam(), loss='categorical_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])


In [ ]:
history_cnn = cnn_model.fit(train_generator, epochs=20, validation_data=val_generator)

In [ ]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, label='Training accuracy')
    plt.plot(epochs, val_acc, label='Validation accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, label='Training loss')
    plt.plot(epochs, val_loss, label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.legend()

In [ ]:
test_loss_tl, test_acc_tl, test_precision_tl, test_recall_tl = cnn_model.evaluate(test_generator)
print(f'Test Loss: {test_loss_tl:.4f}')
print(f'Test Accuracy: {test_acc_tl:.4f}')
print(f'Test Precision: {test_precision_tl:.4f}')
print(f'Test Recall: {test_recall_tl:.4f}')


In [ ]:
##transfer learning

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dropout, BatchNormalization

In [ ]:
train_datagen_tl = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen_tl = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator_tl = train_datagen_tl.flow_from_directory(train_dir, target_size=(128, 128),
                                                          batch_size=32, class_mode='categorical',
                                                          subset='training', shuffle=True, seed=1)
val_generator_tl = train_datagen_tl.flow_from_directory(train_dir, target_size=(128, 128),
                                                        batch_size=32, class_mode='categorical',
                                                        subset='validation', shuffle=True, seed=1)
test_generator_tl = test_datagen_tl.flow_from_directory(test_dir, target_size=(128, 128),
                                                        batch_size=32, class_mode='categorical',
                                                        shuffle=False)

In [ ]:
# 2.5: Build the Transfer Learning Model
base_model = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

In [ ]:
model_tl = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.2),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.1),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.1),
    Dense(3, activation='softmax')
])

In [ ]:
# 2.6: Compile the model
model_tl.compile(optimizer=Adam(), loss='categorical_crossentropy',
                 metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])


In [ ]:
# 2.7: Train the model with EarlyStopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history_tl = model_tl.fit(train_generator_tl, epochs=20, validation_data=val_generator_tl, callbacks=[early_stopping])


In [ ]:
# 2.8: Evaluate the model on test data
test_loss_tl, test_acc_tl, test_precision_tl, test_recall_tl = model_tl.evaluate(test_generator_tl)
print(f'Test Loss: {test_loss_tl:.4f}')
print(f'Test Accuracy: {test_acc_tl:.4f}')
print(f'Test Precision: {test_precision_tl:.4f}')
print(f'Test Recall: {test_recall_tl:.4f}')

# 2.9: Plot Train vs Validation Loss and Accuracy
plot_history(history_tl)